In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Embedding, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping

In [2]:
full_df = pd.read_csv('data/affcon_final.csv')

In [3]:
full_df.head()

,Input.convo_id,Input.train_test_val,Input.msg_id,Input.timestamp,Input.full_text,affcon_gamemove,affcon_reasoning,affcon_rapport,affcon_shareinformation,Input.speaker,...,Input.relative_message_index,Input.year,Input.game_score_speaker,Input.game_score_receiver,Input.game_score_delta,Input.deception_quadrant,Input.num_words,Input.num_characters,Input.sno,Input.sno1
0,Game7-turkey-austria,Train,Game7-turkey-austria-9,197,Im moving my fleet to Alb not for Greece but f...,1,1,1,1,austria-Game7,...,9,1901,3,3,0,Straightforward,12,56,32,16.0
1,Game10-england-russia,Train,Game10-england-russia-36,722,If you take action in the spring to fight Aust...,1,1,1,1,england-Game10,...,36,1905,7,7,0,Straightforward,28,150,100,12.0
2,Game6-england-germany,Train,Game6-england-germany-86,469,This is to help prevent the ottoman advance.,0,1,0,1,england-Game6,...,86,1909,6,9,3,Straightforward,8,44,30,5.0
3,Game11-austria-italy,Validation,Game11-austria-italy-5,45,"And yes I would like peace on our front, I cou...",1,1,1,0,austria-Game11,...,5,1901,3,3,0,Straightforward,31,132,46,22.0
4,Game6-germany-russia,Train,Game6-germany-russia-3,37,"Just to be clear, England is not necessarily a...",1,0,0,1,russia-Game6,...,3,1901,4,5,1,Cassandra,27,148,63,21.0


In [4]:
full_df = full_df.dropna() # dataset contains NaN values, dropping NaNs here

X = full_df['Input.full_text']

full_df["Input.deception_quadrant"] = full_df["Input.deception_quadrant"].apply(lambda x : 1 if x == "Straightforward" else 0)
y = full_df['Input.deception_quadrant']

le = LabelEncoder() # this can convert our categories into labels, make sure you don't have NaNs or Nulls in your data first
y = le.fit_transform(y)
print(y.shape)

# we reshape 
y = y.reshape(-1,1) # the -1 allows it to have whatever number went in there
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(13493,)
(13493, 1)
(10794,)
(10794, 1)
(2699,)
(2699, 1)


In [5]:
max_words = 1000
max_len = 100

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)

sequences = tok.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences,maxlen=max_len)
# X_train = sequence.pad_sequences(sequences)

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## LSTM Model

In [7]:
Inp = Input(name='inputs',shape=[max_len])
x = Embedding(max_words,50,input_length=max_len)(Inp)
x = LSTM(64,name='LSTM_01')(x)
x = Dropout(0.5,name='Dropout')(x)
x = Dense(128,activation='relu',name='Dense_01')(x)
# x = Dropout(0.5,name='Dropout')(x)
out = Dense(1,activation='sigmoid', name='output')(x)

model = Model(inputs=Inp,outputs=out)

model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['acc',f1_m,precision_m, recall_m])

model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           50000     
_________________________________________________________________
LSTM_01 (LSTM)               (None, 64)                29440     
_________________________________________________________________
Dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
Dense_01 (Dense)             (None, 128)               8320      
_________________________________________________________________
output (Dense)               (None, 1)                 129       
Total params: 87,889
Trainable params: 87,889
Non-trainable params: 0
__________________________________________________

In [8]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

model.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
68/68 [==============================] - 1s 16ms/step - loss: 0.2320 - acc: 0.9467 - f1_m: 0.9722 - precision_m: 0.9497 - recall_m: 0.9966 - val_loss: 0.2109 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - val_recall_m: 1.0000
Epoch 2/15
68/68 [==============================] - 1s 13ms/step - loss: 0.2055 - acc: 0.9497 - f1_m: 0.9741 - precision_m: 0.9496 - recall_m: 1.0000 - val_loss: 0.2066 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - val_recall_m: 1.0000
Epoch 3/15
68/68 [==============================] - 1s 14ms/step - loss: 0.2012 - acc: 0.9497 - f1_m: 0.9742 - precision_m: 0.9499 - recall_m: 1.0000 - val_loss: 0.2023 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - val_recall_m: 1.0000
Epoch 4/15
68/68 [==============================] - 1s 12ms/step - loss: 0.1871 - acc: 0.9497 - f1_m: 0.9738 - precision_m: 0.9492 - recall_m: 1.0000 - val_loss: 0.2233 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - v

In [9]:
test_sequences_LSTM = tok.texts_to_sequences(X_test)
X_test_LSTM = sequence.pad_sequences(test_sequences_LSTM,maxlen=max_len)
model.evaluate(X_test_LSTM,y_test)

85/85 [==============================] - 0s 4ms/step - loss: 0.2294 - acc: 0.9470 - f1_m: 0.9726 - precision_m: 0.9474 - recall_m: 1.0000


[0.22943942248821259,
 0.9470174312591553,
 0.9726169109344482,
 0.9474264979362488,
 1.0]

## CNN Model

In [10]:
model_CNN = Sequential(name="CNN_with_embeddings")
model_CNN.add(Embedding(max_words, 50, input_length=max_len))
model_CNN.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
model_CNN.add(MaxPooling1D(pool_size=2))
model_CNN.add(Flatten())
model_CNN.add(Dropout(0.5))
model_CNN.add(Dense(10, activation='relu'))
model_CNN.add(Dense(1, activation='sigmoid'))

model_CNN.compile(loss='binary_crossentropy', 
              optimizer= 'adam',
              metrics=['acc',f1_m,precision_m, recall_m])

model_CNN.summary()

Model: "CNN_with_embeddings"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 50)           50000     
_________________________________________________________________
conv1d (Conv1D)              (None, 93, 32)            12832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 46, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1472)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1472)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                14730     
_________________________________________________________________
dense_1 (Dense)              (None, 1)         

In [11]:
early_stop = EarlyStopping(monitor='val_loss',min_delta=0.00001)

model_CNN.fit(X_train,y_train,
          batch_size=128,
          epochs=15,
          validation_split=0.2,
          callbacks=[early_stop])

Epoch 1/15
68/68 [==============================] - 1s 8ms/step - loss: 0.2798 - acc: 0.9328 - f1_m: 0.9592 - precision_m: 0.9509 - recall_m: 0.9806 - val_loss: 0.2052 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - val_recall_m: 1.0000
Epoch 2/15
68/68 [==============================] - 0s 5ms/step - loss: 0.2003 - acc: 0.9497 - f1_m: 0.9741 - precision_m: 0.9496 - recall_m: 1.0000 - val_loss: 0.2044 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - val_recall_m: 1.0000
Epoch 3/15
68/68 [==============================] - 0s 5ms/step - loss: 0.1979 - acc: 0.9497 - f1_m: 0.9742 - precision_m: 0.9500 - recall_m: 1.0000 - val_loss: 0.2047 - val_acc: 0.9477 - val_f1_m: 0.9731 - val_precision_m: 0.9476 - val_recall_m: 1.0000


In [12]:
test_sequences_CNN = tok.texts_to_sequences(X_test)
X_test_CNN = sequence.pad_sequences(test_sequences_CNN,maxlen=max_len)

model_CNN.evaluate(X_test_CNN,y_test)

85/85 [==============================] - 0s 2ms/step - loss: 0.2073 - acc: 0.9470 - f1_m: 0.9726 - precision_m: 0.9474 - recall_m: 1.0000


[0.20725136995315552,
 0.9470174312591553,
 0.9726169109344482,
 0.9474264979362488,
 1.0]